In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
#Parametros de ejecucion del programa
nombreArchivoDataset = "./Datos en Pasos Intermedios/Dataset Reconstruido.csv"
nombreColumnaDuplicadaSalida = "Interested in buying a mobile home policy"
nombreColumnaSalida = "Number of mobile home policies"
nombreColumnaFase = "Phase"
etiquetaDatosEntrenamiento = "Training"
etiquetaDatosPrueba = "Test"

colorTextoVerde = (35, 155, 86)
colorTextoRojo = (176, 58, 46)

In [3]:
def printColouredText(text, foregroundColor = (0, 0, 0), backgroundColor = (255, 255, 255)):
    return "\033[48;2;{};{};{}m\033[38;2;{};{};{}m{}\033[38;2;0;0;0m\033[48;2;255;255;255m".format(backgroundColor[0], backgroundColor[1], backgroundColor[2], foregroundColor[0], foregroundColor[1], foregroundColor[2], text)

def buildCountingPivotTable(dataframeSource, valuesColumn, indexColumn, mainColumnToGroup, secondaryColumnToGroup):
    pivotTableWith2DetailLevels = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = [secondaryColumnToGroup, mainColumnToGroup], aggfunc = 'count')
    pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel = pd.pivot_table(dataframeSource, values = valuesColumn, index = indexColumn, columns = mainColumnToGroup, aggfunc = 'count')
    pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
    pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
    pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
    fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)
    
    suma = fullPivotTable.sum()
    suma.name = "General Total"
    pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = [suma.name])
    pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
    fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])
    
    for column in fullPivotTable.columns:
        fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]
    return fullPivotTable

In [4]:
fullDatasetFile = pd.read_csv(nombreArchivoDataset, sep = ";", decimal = ".")
fullDatasetFile[nombreColumnaDuplicadaSalida] = fullDatasetFile[nombreColumnaSalida].tolist()
fullDatasetFile.replace({nombreColumnaDuplicadaSalida: {0: "No", 1: "Yes"}}, inplace = True)

#Filtrado de informacion para separar el dataset de entrenamiento y el dataset de prueba del dataset completo
trainingDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosEntrenamiento)]
testDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosPrueba)]

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDataset, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(fullDatasetFile.index)) + " dato(s)", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(trainingDatasetFile.index)) + " dato(s) de entrenamiento", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(testDatasetFile.index)) + " dato(s) de prueba", colorTextoVerde))

Se ha extraído la información del archivo ./Datos en Pasos Intermedios/Dataset Reconstruido.csv
Se tiene(n) 9822 dato(s)
Se tiene(n) de 5822 dato(s) de entrenamiento
Se tiene(n) de 4000 dato(s) de prueba


In [5]:
fullDatasetFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 88 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Customer Subtype                                  9822 non-null   int64 
 1   Number of houses                                  9822 non-null   int64 
 2   Avg size household                                9822 non-null   int64 
 3   Avg age                                           9822 non-null   int64 
 4   Customer main type                                9822 non-null   int64 
 5   Roman catholic                                    9822 non-null   int64 
 6   Protestant                                        9822 non-null   int64 
 7   Other religion                                    9822 non-null   int64 
 8   No religion                                       9822 non-null   int64 
 9   Married                       

In [6]:
pivotTablesList = {}

for nombreColumna in fullDatasetFile.columns:
    if nombreColumna != nombreColumnaSalida and nombreColumna != nombreColumnaDuplicadaSalida and nombreColumna != nombreColumnaFase:
        pivotTablesList[nombreColumna] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, nombreColumna, nombreColumnaDuplicadaSalida, nombreColumnaFase)
        print("pivotTablesList[\"" + nombreColumna + "\"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, \"" + nombreColumna + "\", nombreColumnaDuplicadaSalida, nombreColumnaFase)")
        print("display(pivotTablesList[\"" + nombreColumna + "\"])\n")

#pivotTablesList[] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, col, nombreColumnaDuplicadaSalida, nombreColumnaFase)

#display(pivotTable1)

pivotTablesList["Customer Subtype"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

pivotTablesList["Number of houses"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

pivotTablesList["Avg size household"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg size household", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size household"])

pivotTablesList["Avg age"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

pivotTablesList["Customer main type"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer main type", nombreColumnaDuplicadaSalida, nombreC

pivotTablesList["Purchasing power class"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Purchasing power class", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Purchasing power class"])

pivotTablesList["Contribution private third party insurance"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Contribution private third party insurance", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution private third party insurance"])

pivotTablesList["Contribution third party insurance (firms)"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Contribution third party insurance (firms)", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Contribution third party insurance (firms)"])

pivotTablesList["Contribution third party insurane (agriculture)"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Contribution third party insurane (agriculture)",

pivotTablesList["Number of agricultural machines policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of agricultural machines policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of agricultural machines policies"])

pivotTablesList["Number of moped policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of moped policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of moped policies"])

pivotTablesList["Number of life insurances"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of life insurances", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of life insurances"])

pivotTablesList["Number of private accident insurance policies"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of private accident insurance policies", nombreColumnaDuplicadaSalida, nombreColumnaFase)
displ

In [7]:
pivotTablesList["Customer Subtype"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer Subtype", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer Subtype"])

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        111   13   
2                                                         76    6   
3                                                        224   25   
4                                                         50    2   
5                                                         43    2   
6                                                        107   12   
7                                                         41    3   
8                                                        288   51   
9                                                        266   12   
10                                                       156    9   
11                                                       144    9   
12                                                        95   16   
13                                                       166   13   
15                                                         5    0   
16                                                        16    0   
17                                                         9    0   
18                                                        19    0   
19                                                         3    0   
20                                                        23    2   
21                                                        15    0   
22                                                        94    4   
23                                                       247    4   
24                                                       175    5   
25                                                        80    2   
26                                                        47    1   
27                                                        49    1   
28                                                        25    0   
29                                                        84    2   
30                                                       114    4   
31                                                       199    6   
32                                                       133    8   
33                                                       764   46   
34                                                       173    9   
35                                                       206    8   
36                                                       209   16   
37                                                       122   10   
38                                                       316   23   
39                                                       309   19   
40                                                        71    0   
41                                                       200    5   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                     81   13   192   26   
2                                                     61    5   137   11   
3                                                    176    8   400   33   
4                                                     37    1    87    3   
5                                                     25    0    68    2   
6                                                     76   14   183   26   
7                                                     26    2    67    5   
8                                                    186   21   474   72   
9                                                    177    5   443   17   
10                                                    92   14   248   23   
11                                                   126    7   270   16   
12                                          

In [8]:
pivotTablesList["Number of houses"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Number of houses", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Number of houses"])

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                       4952  315   
2                                                        472   33   
3                                                         39    0   
4                                                          2    0   
5                                                          1    0   
6                                                          1    0   
7                                                          5    0   
8                                                          1    0   
10                                                         1    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                   3437  211  8389  526   
2                                                    290   26   762   59   
3                                                     24    1    63    1   
4                                                      2    0     4    0   
5                                                      2    0     3    0   
6                                                      2    0     3    0   
7                                                      3    0     8    0   
8                                                      1    0     2    0   
10                                                     1    0     2    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested in buying a mobile home policy General Total  
1                                                  8915  
2                                                   821  
3                                                    64  
4                                                     4  
5                                                     3  
6                                                     3  
7                                                     8  
8                                                     2  
10                                                    2  
General Total                                      9822

In [9]:
pivotTablesList["Avg size household"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg size household", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg size household"])

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        276    8   
2                                                       2016  115   
3                                                       2475  171   
4                                                        643   50   
5                                                         64    4   
6                                                          0    0   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                    165    3   441   11   
2                                                   1405   80  3421  195   
3                                                   1763  104  4238  275   
4                                                    391   48  1034   98   
5                                                     35    3    99    7   
6                                                      3    0     3    0   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested in buying a mobile home policy General Total  
1                                                   452  
2                                                  3616  
3                                                  4513  
4                                                  1132  
5                                                   106  
6                                                     3  
General Total                                      9822

In [10]:
pivotTablesList["Avg age"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Avg age", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Avg age"])

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                         73    1   
2                                                       1365   87   
3                                                       2817  183   
4                                                       1009   64   
5                                                        181   12   
6                                                         29    1   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                     30    0   103    1   
2                                                    888   69  2253  156   
3                                                   2034  120  4851  303   
4                                                    663   41  1672  105   
5                                                    128    8   309   20   
6                                                     19    0    48    1   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested in buying a mobile home policy General Total  
1                                                   104  
2                                                  2409  
3                                                  5154  
4                                                  1777  
5                                                   329  
6                                                    49  
General Total                                      9822

In [11]:
pivotTablesList["Customer main type"] = buildCountingPivotTable(fullDatasetFile, nombreColumnaSalida, "Customer main type", nombreColumnaDuplicadaSalida, nombreColumnaFase)
display(pivotTablesList["Customer main type"])

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        504   48   
2                                                        436   66   
3                                                        827   59   
4                                                         52    0   
5                                                        554   15   
6                                                        201    4   
7                                                        530   20   
8                                                       1474   89   
9                                                        625   42   
10                                                       271    5   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                    380   27   884   75   
2                                                    288   37   724  103   
3                                                    577   50  1404  109   
4                                                     27    0    79    0   
5                                                    368    3   922   18   
6                                                    116    5   317    9   
7                                                    316   15   846   35   
8                                                   1069   62  2543  151   
9                                                    411   33  1036   75   
10                                                   210    6   481   11   
General Total                                       3762  238  9236  586   

Phase                                                    
Interested in buying a mobile home policy General Total  
1                                                   959  
2                                                   827  
3                                                  1513  
4                                                    79  
5                                                   940  
6                                                   326  
7                                                   881  
8                                                  2694  
9                                                  1111  
10                                                  492  
General Total                                      9822

In [12]:
pivotTableWith2DetailLevels = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = "Customer Subtype", columns = [nombreColumnaFase, nombreColumnaDuplicadaSalida], aggfunc = 'count')
pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = "Customer Subtype", columns = nombreColumnaDuplicadaSalida, aggfunc = 'count')
pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)

suma = fullPivotTable.sum()
suma.name = "General Total"
pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = [suma.name])
pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])

for column in fullPivotTable.columns:
    fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]

#x1.info()
display(fullPivotTable)

Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        111   13   
2                                                         76    6   
3                                                        224   25   
4                                                         50    2   
5                                                         43    2   
6                                                        107   12   
7                                                         41    3   
8                                                        288   51   
9                                                        266   12   
10                                                       156    9   
11                                                       144    9   
12                                                        95   16   
13                                                       166   13   
15                                                         5    0   
16                                                        16    0   
17                                                         9    0   
18                                                        19    0   
19                                                         3    0   
20                                                        23    2   
21                                                        15    0   
22                                                        94    4   
23                                                       247    4   
24                                                       175    5   
25                                                        80    2   
26                                                        47    1   
27                                                        49    1   
28                                                        25    0   
29                                                        84    2   
30                                                       114    4   
31                                                       199    6   
32                                                       133    8   
33                                                       764   46   
34                                                       173    9   
35                                                       206    8   
36                                                       209   16   
37                                                       122   10   
38                                                       316   23   
39                                                       309   19   
40                                                        71    0   
41                                                       200    5   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                     81   13   192   26   
2                                                     61    5   137   11   
3                                                    176    8   400   33   
4                                                     37    1    87    3   
5                                                     25    0    68    2   
6                                                     76   14   183   26   
7                                                     26    2    67    5   
8                                                    186   21   474   72   
9                                                    177    5   443   17   
10                                                    92   14   248   23   
11                                                   126    7   270   16   
12                                          